In [1]:
"""
Name: con_example.ipynb
Authors: Christian Haack, Stephan Meighen-Berger
Example of the realistic simulations that can be done
"""
%load_ext autoreload
%autoreload 2

In [3]:
 # General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import yaml
from copy import deepcopy
from cProfile import Profile
import networkx as nx
# picture path
PICS = '../pics/'
# Module imports
from contagion import Contagion, config
from contagion.config import _baseconfig
from distributed import client
from itertools import product
from distributed import Client, performance_report, progress



In [4]:
my_config = yaml.safe_load(open("param_scan_config.yaml"))

In [5]:
class ParScan(object):
    def __init__(self):
        self._par_range = None
        
    def par_update(self, val):
        pass
            
    def make_generator(self, configs):
        for conf in configs:
            for par in self._par_range:
                yield self.par_update(par, conf)
        

class ScanMu(ParScan):
    def __init__(self):
        super().__init__()
        self._par_range = [0.01, 0.05, 0.1, 0.2, 0.5, 0.7]
        self.par_name = "Mu"
        self.title = "Fraction of external connections"
        
    def par_update(self, par, conf):
        conf = deepcopy(conf)        
        conf["population"]["nx"]["kwargs"]["mu"] = par
        return conf
    
class ScanScenario(ParScan):
    def __init__(self):
        super().__init__()
        
        self.t_dur_initial = 28
        t_starts = [7, 14, 28, 35]
        
        t_durs_soft = [30, 3*30, 6*30, 9*30, 12*30, 14*30]
        hard_scalings = [1/1.78, 2/7.8]
        soft_scalings = [2/7.8, 3/7.8, 5/7.8]
        
        self._par_range = product(
            t_starts,
            t_durs_soft,
            hard_scalings,
            soft_scalings)
        self.par_name = "TStart"
        self.title = "Lockdown Start"
        
    def par_update(self, par, conf):
        t_start, t_dur_soft, hard_scaling, soft_scaling = par
        conf = deepcopy(conf)        
        conf["population"]["nx"]["kwargs"]["mu"] = par
        conf["scenario"]["class"] = "SocialDistancing"
        conf["scenario"]["t_steps"] = [
            t_start,
            t_start + self.t_dur_initial,
            t_start + self.t_dur_initial + t_dur_soft]
        conf["scenario"]["contact_rate_scalings"] = [hard_scaling, soft_scaling, 1]   
        return conf

In [40]:
scan = ScanScenario()
gen = scan.make_generator([my_config])

scan2 = ScanMu()
gen2 = scan2.make_generator(gen)


In [39]:
def submit_func(conf):
    contagion = Contagion(conf)
    if hasattr(contagion.pop, "_graph"):
        g = deepcopy(contagion.pop._graph)
    else:
        g = None
    contagion.sim()
    stats = pd.DataFrame(contagion.statistics)
    
    inf_per_day = []
    
    for day in contagion.trace_contacts:
        inf_per_day.append(np.unique(day))
    
    return (stats, inf_per_day)
    

In [33]:
client = Client(scheduler_file="scheduler.json")
#client.restart()

In [41]:
futures = []
for conf in gen2:
    futures.append(client.submit(submit_func, conf, pure=False))


In [23]:
progress(futures, notebook=True)

VBox()

In [42]:
with performance_report(filename="dask-report.html"):
    results = client.gather(futures)

KilledWorker: ('submit_func-4ab395b9-cd28-4e61-9225-880f44f06129', <Worker 'tcp://10.152.133.48:46713', name: 78, memory: 0, processing: 363>)

In [43]:
futures

[<Future: lost, type: builtins.tuple, key: submit_func-fb5e2eff-62fc-44ec-b3c7-74ccf65357a6>,
 <Future: lost, type: builtins.tuple, key: submit_func-ae50516d-084a-4350-a5eb-a150e2853650>,
 <Future: lost, type: builtins.tuple, key: submit_func-d29a630b-2c1a-426f-a956-0580c25c1547>,
 <Future: lost, type: builtins.tuple, key: submit_func-fa226fe1-f62c-4b79-8864-481d5cd0f28c>,
 <Future: lost, type: builtins.tuple, key: submit_func-b822ea3f-5b18-4561-9fd9-1502d4d8aad3>,
 <Future: lost, type: builtins.tuple, key: submit_func-4813d0e6-5e35-4372-abd8-54baf039d476>,
 <Future: lost, type: builtins.tuple, key: submit_func-d8beb98a-0dc4-442f-90b6-0a87f8ed38cd>,
 <Future: lost, type: builtins.tuple, key: submit_func-9aadce7e-0e12-4773-ae5a-60bc157b8e8d>,
 <Future: lost, type: builtins.tuple, key: submit_func-52fd1a16-4c2d-455c-bf34-1c0b938e63aa>,
 <Future: lost, type: builtins.tuple, key: submit_func-ba00544b-3cc0-427e-9319-a48932feab46>,
 <Future: lost, type: builtins.tuple, key: submit_func-9b43b

In [39]:
traceback.print_tb(res.traceback(), )

  File "<ipython-input-22-da54d23c0432>", line 2, in submit_func
    contagion = Contagion(conf)
  File "/dss/dsshome1/lxc05/ge79juv2/software/scripts/contagion/contagion/contagion.py", line 154, in __init__
    self.pop = population_class()
  File "/dss/dsshome1/lxc05/ge79juv2/software/scripts/contagion/contagion/population.py", line 1009, in __init__
    self._graph = gen_func(
  File "/dss/dsshome1/lxc05/ge79juv2/software/scripts/contagion/contagion/population.py", line 892, in lfr_ba
    weights=node_degrees[candidates])[0]


In [33]:
client

Client Scheduler: tcp://10.156.79.101:43687 Dashboard: http://10.156.79.101:8787/status,Cluster Workers: 1 Cores: 1 Memory: 1.80 GB
